# Генерация заголовков новостей с использованием Word Embeddings
За основу взята реализация одного из заданий на курсе [Udacity Deep Learning Foundation](https://www.udacity.com/course/deep-learning-nanodegree-foundation--nd101). 

In [106]:
import helper # Вспомогательные функции для сохранения и загрузки модели и параметров
import numpy as np
import random
import time
import tensorflow as tf
from tensorflow.contrib import seq2seq
from collections import Counter

Здесь задаем, на каких данных будем обучать сеть

In [107]:
# Все статьи
mode = "full"       

# Выборка (~800 статей)
# mode = "sample"  

Откроем файл с предварительно обработанным текстом (см. ноутбук preprocessing)

In [108]:
data_dir = './data/headers_' + mode + '.txt'
text = helper.load_data(data_dir)

## Данные
Посмотрим, что внутри текста:

In [109]:
# Диапазон номеров выводимых заголовков
view_sentence_range = (0, 10)

print('Немного статистики')
print('Уникальных слов: {}'.format(len({word: None for word in text.split(" ")})))
sentences = [sentence for sentence in text.split('. ')]
print('Количество заголовков: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Среднее количество слов в заголовке: {}'.format(np.average(word_count_sentence)))

print()
print('Заголовки с {} по {}:'.format(*view_sentence_range))
print('\n'.join(text.split('. ')[view_sentence_range[0]:view_sentence_range[1]]))

Немного статистики
Уникальных слов: 80342
Количество заголовков: 58604
Среднее количество слов в заголовке: 7.871851750733739

Заголовки с 0 по 10:
рпцз призвала вынести ленина из мавзолея и начать декоммунизацию
найдены тела пропавших моряков с американского эсминца
полиция мэриленда арестовала подозреваемых в убийстве россиянина зиберова
концерт рэпера басты в одессе отменили после угроз украинских националистов
более 200 тысяч человек выступили против презумпции доверия к полицейским
мутко раскритиковал газзаева за ответ путину во время прямой линии
у меня не железные яйца
ковалев прокомментировал второе подряд поражение от уорда
в фсвтс прокомментировали поставки корабельных вертолетов ка-52к в египет
украинские националисты отобрали флаг лгбт у участников гей-парада в киеве


## Предварительная обработка

### Создание словарей
Чтобы создать word embedding, нужно преобразовать слова в числовые айдишники. Функция ниже создает и возвращает два словаря:
- `vocab_to_int`: Слово -> ID 
- `int_to_vocab`: ID -> Слово

`text` - Текст со всеми заголовками, разделенный на слова

In [110]:
def create_lookup_tables(text):

    counter = Counter(text)
    vocab = sorted(counter, key=counter.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    return vocab_to_int, int_to_vocab

### Токенизация знаков препинания
Мы будем разделять текст на массив слов, используя пробел в качестве разделителя. При этом в исходном тексте знаки пунктуации в общем случае "приклеены" к слову (не отделены пробелами), поэтому нейронной сети будет сложно отличить слово "привет" от "привет!". 

Чтобы предотвратить такие случаи, заменим имеющиеся в тексте знаки пунктуации на отдельные токены. 

_(Большая часть знаков уже была удалена в ходе препроцессинга, но я решил не менять этот код, оставив его таким, какой он был в изначальном варианте.)_

Этот словарь будет использоваться для преобразования знаков препинания в токены, добавляя пробел перед ними.

In [111]:
def token_lookup():
    tokens = {
        ".": "||PERIOD||",
        ",": "||COMMA||",
        '"': "||QUOT_MARK||",
        ";": "||SEMICOL||",
        "!": "||EXCL_MARK||",
        "?": "||QUEST_MARK||",
        "(": "||L_PARENTH||",
        ")": "||R_PARENTH||",
        "--": "||DASH||",
        "\n": "||RETURN||"
    }
    
    return tokens

Обработаем текст и сохраняем его, как контрольную точку

In [112]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables, text)

Загружаем текст и словари

In [113]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

Ради любопытства взглянем на какой-нибудь из словарей

In [114]:
int_to_vocab

{0: '||period||',
 1: 'в',
 2: 'на',
 3: 'с',
 4: 'о',
 5: 'и',
 6: 'за',
 7: 'по',
 8: 'россии',
 9: 'из',
 10: 'сша',
 11: 'для',
 12: 'от',
 13: 'из-за',
 14: 'к',
 15: 'об',
 16: '||comma||',
 17: 'у',
 18: 'после',
 19: 'рассказал',
 20: 'назвал',
 21: 'сми',
 22: 'трампа',
 23: 'во',
 24: 'москве',
 25: 'под',
 26: 'путин',
 27: 'видео',
 28: 'рублей',
 29: 'при',
 30: 'против',
 31: 'не',
 32: 'до',
 33: 'долларов',
 34: 'украины',
 35: 'сети',
 36: 'россиян',
 37: 'суд',
 38: 'сирии',
 39: 'российских',
 40: 'предложили',
 41: 'тысяч',
 42: 'человек',
 43: 'глава',
 44: 'назвали',
 45: 'трамп',
 46: 'года',
 47: 'порошенко',
 48: 'предложил',
 49: 'рассказали',
 50: 'время',
 51: 'умер',
 52: 'нашли',
 53: 'сообщили',
 54: 'россию',
 55: 'области',
 56: 'со',
 57: 'россия',
 58: 'путина',
 59: 'заявил',
 60: 'его',
 61: 'без',
 62: 'миллионов',
 63: 'лет',
 64: 'украине',
 65: 'власти',
 66: 'мира',
 67: 'задержали',
 68: 'иг',
 69: 'два',
 70: 'российский',
 71: 'подмосковье',

## Строим нейронную сеть

### Входные данные
Функция `get_inputs()` создает набор TensorFlow Placeholder для нейронной сети:
- Плейсхолдер с названием `input` для входного текста, используя параметр `name` для [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder);
- Плейсхолдер для целевых значений
- Плейсхолдер для Learning Rate

In [115]:
def get_inputs():
    inputs = tf.placeholder(tf.int32, shape=[None, None], name="input")
    targets = tf.placeholder(tf.int32, shape=[None, None], name="targets")
    l_rate = tf.placeholder(tf.float32, name="learning_rate")
    
    return inputs, targets, l_rate

### Строим и инициализируем рекуррентные ячейки
Заполняем [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell) одним или несколькими [`BasicLSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell).
- `rnn_size` задает количество ячеек;
- `num_layers` задает количество рекуррентных слоев;
- функция [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) из MultiRNNCell инициализирует начальное состояние;
- `get_init_cell` возвращает набор слоев в виде MultiRNNCell и начальное состояние.

In [116]:
def get_init_cell(batch_size, rnn_size, num_layers=2, dropout=0.1):
    basic_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    basic_cell_with_dropout = tf.contrib.rnn.DropoutWrapper(basic_cell, output_keep_prob=(1-dropout))
    multi_rnn_cell = tf.contrib.rnn.MultiRNNCell([basic_cell_with_dropout] * num_layers)
    init_state = tf.identity(multi_rnn_cell.zero_state(batch_size, tf.float32), name="initial_state")

    return multi_rnn_cell, init_state

### Word Embedding
Собственно, создание Word Embedding. Функция возвращает эмбеддинг для `input_data`.

In [117]:
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    
    return tf.nn.embedding_lookup(embedding, input_data)

### Строим рекуррентный слой
Из ячейки, созданной с помощью `get_init_cell()` строим рекуррентный слой. Функция выдает выходные значения после прогона данных через RNN, и состояние слоя.

In [118]:
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    
    return outputs, final_state

### Собираем сеть
Применяем реализованные выше функции:
- Создаем эмбеддинги из `input_data`, используя `get_embed(input_data, vocab_size, embed_dim)`;
- Передаем из в рекуррентный слой `build_rnn(cell, inputs)`;
- Применяем fully connected слой с линейной активацией, передавая результаты из рекуррентного слоя и `vocab_size` в качестве размерности выходного значения.

В результате возвращаются logits и финальное состояние сети.

In [119]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    
    inputs = get_embed(input_data, vocab_size, rnn_size)
    outputs, final_state = build_rnn(cell, inputs)
    logits = tf.contrib.layers.fully_connected(outputs, num_outputs=vocab_size, activation_fn=None)
    
    return logits, final_state

### Батчи
Функция `get_batches` создает батчи из входного текста `int_text`.  Каждый батч - `Numpy array` с размерностями `(количество батчей, 2, размер батча, длина последовательности)`. Каждый батч содержит два элемента:
- Первый элемент - батч входных значений размерности `[размер батча, длина последовательности]`
- Второй элемент - батч целевых значений размерности `[размер батча, длина последовательности]`

Если для последнего батча недостаточно данных, отбрасываем его.

Например, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` вернет следующий массив:
```
[
  # Первый батч
  [
    # Батч входных значений
    [[ 1  2  3], [ 7  8  9]],
    # Батч целевых значений
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Второй батч
  [
    # Батч входных значений
    [[ 4  5  6], [10 11 12]],
    # Батч целевых значений
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [120]:
def get_batches(int_text, batch_size, seq_length):

    num_batches = len(int_text) // (batch_size * seq_length)
    batches = []
    
    for batch_idx in range(num_batches):
        inputs=[]
        targets=[]
        
        for seq_idx in range(batch_size):
            i = batch_idx * seq_length + seq_idx * seq_length
            inputs.append(int_text[i:i+seq_length])
            targets.append(int_text[i+1: i+seq_length+1])
        
        batches.append([inputs, targets])
    
    return np.array(batches)

## Вспомогательные функции
### Загрузка тензоров
Загружаем тензоры из `loaded_graph`, используя [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name). 

In [121]:
def get_tensors(loaded_graph):

    inputs = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    
    return inputs, initial_state, final_state, probs

### Выбор слова
Функция `pick_word()` выбирает следующее слово на основе вероятностей в `probabilities`.

In [122]:
def pick_word(probabilities, int_to_vocab):

    return int_to_vocab[np.argmax(probabilities)]

### Список первых слов
Составляем список первых слов заголовков. Будем использовать их в качестве входного значения для генерации заголовка.

In [123]:
first_words = list(set([line.split(" ")[0] for line in text.split('. ')]))

### Генерация заголовка
Функция загружает граф, восстанавливает сессию и возвращает сгенерированную последовательность слов. Параметры:
- `get_length`: длина генерируемой последовательности (сколько слов нужно сгенерировать);
- `prime_word`: начальное (входное) слово для генерации;
- `load_dir`: первая часть названия файла с графом;
- `rnn_layers`: количество слоев (используется только для построения имени файла);
- `rnn_size`: размер слоев (используется только для построения имени файла).

In [124]:
def infer(gen_length, prime_word, load_dir, rnn_layers, rnn_size):
    
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        
        # Загружаем модель
        loader = tf.train.import_meta_graph(load_dir +'_{}_{}.meta'.format(rnn_layers, rnn_size) )
        loader.restore(sess, load_dir +'_{}_{}'.format(rnn_layers, rnn_size))

        # Получаем тензоры из модели
        input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

        # Инициализируем переменную, где будем хранить сгенерированную последовательность
        gen_sentences = [prime_word]
    
        prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

        # Генерация последовательности
        for n in range(gen_length):
            
            dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
            dyn_seq_length = len(dyn_input[0])
    
            # Получаем вероятности
            probabilities, prev_state = sess.run(
                [probs, final_state],
                {input_text: dyn_input, initial_state: prev_state})
    
            # Получаем следующее слово
            pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
            gen_sentences.append(pred_word)

        # Удаляем токены пунктуации, заменяя их на соответствующие символы
        headlines = ' '.join(gen_sentences)
        for key, token in token_dict.items():
            ending = ' ' if key in ['\n', '(', '"'] else ''
            headlines = headlines.replace(' ' + token.lower(), key)
        headlines = headlines.replace('\n ', '\n')
        headlines = headlines.replace('( ', '(')
        headlines = headlines.replace('. ', '.\n')
                
        print(headlines)
        return headlines

## Обучение нейронной сети
### Гиперпараметры

In [125]:
# Количество эпох (итераций обучения)
num_epochs = 20

# Размер батча
batch_size = 32

# Размер рекуррентного слоя
# rnn_size = 64

# Длина последовательности в батче, определяет как далеко "назад" сеть должна помнить контекст
seq_length = 10

# Скорость обучения
learning_rate = 0.01

# Выводить статистику через каждые N батчей
show_every_n_batches = 500

# Длина генерируемой последовательности
gen_length = 100

# массив опций для обучения; используется для того, чтобы определить наиболее удачную конфигурацию нейронной сети; 
# для обучения только по одному набору параметров, можно закомментировать остальные наборы
options = [
#     {
#         'rnn_size': 32,
#         'rnn_layers': 1
#     },
    {
        'rnn_size': 64,
        'rnn_layers': 1
    },
#     {
#         'rnn_size': 32,
#         'rnn_layers': 2
#     },
#     {
#         'rnn_size': 64,
#         'rnn_layers': 2
#     }
]

# Первая часть названия файла модели
save_dir = './models/word_emb_'

## Обучение
Фиксируем первое слово для всех итераций, чтобы сравнивать результаты генерации. Затем проходимся по всем конфигурациям и обучаем сеть, попутно выводя промежуточные результаты и сохраняя их в файл.

In [126]:
prime_word = first_words[random.randint(0, len(first_words))]

for option in options:

    option['start'] = time.time()
    print('Обучение модели Word Embeddings с параметрами (слои: {}, размер: {})'.format(option['rnn_layers'], option['rnn_size']))
    
    with open('results_word_emb_{}_{}.txt'.format(option['rnn_layers'], option['rnn_size']), 'a') as file:
        train_graph = tf.Graph()
        
        # Строим граф вычислений
        with train_graph.as_default():
            vocab_size = len(int_to_vocab)
            input_text, targets, lr = get_inputs()
            input_data_shape = tf.shape(input_text)
            cell, initial_state = get_init_cell(input_data_shape[0], option['rnn_size'], option['rnn_layers'])
            logits, final_state = build_nn(cell, option['rnn_size'], input_text, vocab_size)

            # Вычисляем вероятности для слов
            probs = tf.nn.softmax(logits, name='probs')

            # Функция потерь
            cost = seq2seq.sequence_loss(
                logits,
                targets,
                tf.ones([input_data_shape[0], input_data_shape[1]]))

            # Функция оптимизация
            optimizer = tf.train.AdamOptimizer(lr)
            # На мой взгляд, Adam выдает чуть более качественные результаты, чем RMSProp,
            # но вы можете попробовать и его:
            # optimizer = tf.train.RMSPropOptimizer(lr)

            # Вычисляем градиенты
            gradients = optimizer.compute_gradients(cost)
            capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
            train_op = optimizer.apply_gradients(capped_gradients)
        
        # Создаем батчи
        batches = get_batches(int_text, batch_size, seq_length)

        # Запускаем обучение
        with tf.Session(graph=train_graph) as sess:
            sess.run(tf.global_variables_initializer())
            
            # Итерация по эпохам
            for epoch_i in range(num_epochs):
                state = sess.run(initial_state, {input_text: batches[0][0]})
                
                # Проходимся по всем батчам
                for batch_i, (x, y) in enumerate(batches):
                    
                    # Задаем входные и целевые данные, состояние и скорость обученя
                    feed = {
                        input_text: x,
                        targets: y,
                        initial_state: state,
                        lr: learning_rate}
                    
                    # Отправляем все это добро на обучение
                    train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

                    # Для красоты и удобства считаем, сколько еще времени осталось на обучение
                    t = time.time()
                    time_diff = t - option['start']
                    rem_batches = num_epochs * len(batches) - (epoch_i * len(batches) + batch_i)
                    total_time = round((num_epochs * len(batches) / (epoch_i * len(batches) + batch_i + 1)) * time_diff)
                    rem_time = round(total_time - time_diff)
                    m, s = divmod(rem_time, 60)
                    h, m = divmod(m, 60)
                    
                    # Выводим статистику каждые <show_every_n_batches> батчей
                    if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                        print(' ' * 100, end='\r')
                        print('Итерация {:>3} Батч {:>4}/{}   train_loss = {:.3f}'.format(
                            epoch_i,
                            batch_i,
                            len(batches),
                            train_loss))
                    
                    print("Оценка оставшегося времени на обучение по текущему набору параметров: %d:%02d:%02d " % (h, m, s), end="\r")
                
                # Сохраняем модель
                saver = tf.train.Saver()
                saver.save(sess, save_dir + "_{}_{}".format(option['rnn_layers'], option['rnn_size']))
                print()
                
                # Смотрим, что получилось в этой итерации
                print("Сгенерированный текст для итерации {}:".format(epoch_i))
                generated_text = infer(gen_length, prime_word, save_dir, option['rnn_layers'], option['rnn_size'])

                # И также сохраняем в файл
                file.write('Итерация ' + str(epoch_i) + '\n')
                file.write(generated_text)

            # В конце считаем время, затраченное на обучение по текущему набору параметров
            time_diff = time.time() - option['start']
            m, s = divmod(time_diff, 60)
            h, m = divmod(m, 60)
            
            print("Затраченное время: %d:%02d:%02d " % (h, m, s))
            file.write("Total time for training this option: %d:%02d:%02d " % (h, m, s))
            print('Обучение модели Word Embeddings с параметрами {} {} завершено'.format(option['rnn_layers'], option['rnn_size']))

Обучение модели Word Embeddings с параметрами (слои: 1, размер: 64)
Итерация   0 Батч    0/1630   train_loss = 11.125                                                   
Итерация   0 Батч  500/1630   train_loss = 3.225                                                    
Итерация   0 Батч 1000/1630   train_loss = 3.111                                                    
Итерация   0 Батч 1500/1630   train_loss = 3.384                                                    
Оценка оставшегося времени на обучение по текущему набору параметров: 9:25:25 
Сгенерированный текст для итерации 0:
INFO:tensorflow:Restoring parameters from ./models/word_emb__1_64
тома.
попал в сети во вселенной.
тесак получил 10 из сша.
трех отсутствие антивещества во вселенной.
тесак получил 10 10 лет колонии.
макрон пригласил трампа в париж.
распространители вируса petya просчитались от поездки на прощание с колем.
минимальный набор продуктов с шантажом.
сша заявили о связях советника трампа в париж.
распространители